# Aufgabe 2 - CNN


In [ ]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
import numpy as np


## Variablen

In [ ]:
epochs = 20
batch_size = 128
sgd = SGD(0.01)


## Dataset

In [ ]:
# Grab the MNIST dataset
# The dataset is already split into train and test set (with distribution train:60k to test:10k)
print("[INFO] accessing MNIST...")
((train_X, train_y), (test_X, test_y)) = mnist.load_data()

# Each image in the MNIST dataset is represented as a 28x28x1
# Flatten the images from 28x28 pixels to list with length 784 to be able to apply it to standart feed forward neural network
train_X = train_X.reshape((train_X.shape[0], 28 * 28 * 1))
test_X = test_X.reshape((test_X.shape[0], 28 * 28 * 1))

# Scale data to the range of [0, 1]
train_X = train_X.astype("float32") / 255.0
test_X = test_X.astype("float32") / 255.0

# Perform one-hot-encoding to convert the labels from integers to vectors
lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.transform(test_y)


## a) Netzwerk


In [ ]:
# define the 784-256-128-10 architecture using Keras
model = Sequential()
model.add(Dense(256, input_shape=(784,), activation="sigmoid"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(10, activation="softmax"))


# train the model using SGD
print("[INFO] training network...")
epochs = 20
sgd = SGD(0.01)
model.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])
H = model.fit(train_X, train_y, validation_data=(test_X, test_y),
	epochs=epochs, batch_size=128, verbose=1)


## b) Training


In [ ]:
epochs = 20
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(test_X, batch_size=128)
print(classification_report(test_y.argmax(axis=1),
	predictions.argmax(axis=1),
	target_names=[str(x) for x in lb.classes_]))


## d)


In [ ]:
# plot the training accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, epochs), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label="val_acc")
plt.title("Training Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend()
#plt.savefig(args["output"])
plt.show()

# plot the training loss

plt.plot(np.arange(0, epochs), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, epochs), H.history["val_loss"], label="val_loss")
plt.title("Training Loss")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
#plt.savefig(args["output"])
plt.show()


Batch normalization ist die Anwendung einer Normalisierungstechnik auf die Eingabewerte, also die Input-Vektoren, der hidden layers. Grundsätzlich wird batch normalization verwendet um die Konvergenzrate und damit die Geschwindigkeit beim Training zu erhöhen, indem der covariate shift reduziert wird.
